<a href="https://colab.research.google.com/github/ritika-dhanda/Movies-Recommendation-system/blob/main/Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

# Load the datasets
movies = pd.read_csv("https://s3-us-west-2.amazonaws.com/recommender-tutorial/movies.csv")
ratings = pd.read_csv("https://s3-us-west-2.amazonaws.com/recommender-tutorial/ratings.csv")

# Create user-item matrix
def create_user_item_matrix(df):
    N = df['userId'].nunique()
    M = df['movieId'].nunique()

    user_mapper = dict(zip(np.unique(df["userId"]), list(range(N))))
    movie_mapper = dict(zip(np.unique(df["movieId"]), list(range(M))))

    user_inv_mapper = dict(zip(list(range(N)), np.unique(df["userId"])))
    movie_inv_mapper = dict(zip(list(range(M)), np.unique(df["movieId"])))

    user_index = [user_mapper[i] for i in df['userId']]
    movie_index = [movie_mapper[i] for i in df['movieId']]

    X = csr_matrix((df["rating"], (movie_index, user_index)), shape=(M, N))

    return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper

X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper = create_user_item_matrix(ratings)

# Find similar movies using KNN
def find_similar_movies(movie_id, X, movie_mapper, movie_inv_mapper, k=10):
    movie_ind = movie_mapper[movie_id]
    movie_vec = X[movie_ind]
    kNN = NearestNeighbors(n_neighbors=k+1, algorithm="auto", metric='cosine')
    kNN.fit(X)
    movie_vec = movie_vec.reshape(1, -1)
    neighbour = kNN.kneighbors(movie_vec, return_distance=False)
    neighbour_ids = [movie_inv_mapper[n] for n in neighbour.flatten()[1:]]
    return neighbour_ids

# Recommend similar movies based on user input
def recommend_movies_based_on_input(movie_title, X, movies, movie_mapper, movie_inv_mapper, k=10):
    # Find the movie ID based on the input title
    movie_id = movies[movies['title'].str.contains(movie_title, case=False, na=False)]

    if movie_id.empty:
        print("Movie not found. Please try another title.")
        return

    movie_id = movie_id.iloc[0]['movieId']  # Get the first matching movie's ID
    similar_movie_ids = find_similar_movies(movie_id, X, movie_mapper, movie_inv_mapper, k)

    print(f"\nMovies similar to '{movie_title}':")
    for movie_id in similar_movie_ids:
        movie_title = movies[movies['movieId'] == movie_id]['title'].values[0]
        print(f"- {movie_title}")

# Take user input for the movie title
user_input = input("Enter a movie you like: ")
recommend_movies_based_on_input(user_input, X, movies, movie_mapper, movie_inv_mapper, k=10)


Enter a movie you like: Toy Story

Movies similar to 'Toy Story':
- Toy Story 2 (1999)
- Jurassic Park (1993)
- Independence Day (a.k.a. ID4) (1996)
- Star Wars: Episode IV - A New Hope (1977)
- Forrest Gump (1994)
- Lion King, The (1994)
- Star Wars: Episode VI - Return of the Jedi (1983)
- Mission: Impossible (1996)
- Groundhog Day (1993)
- Back to the Future (1985)
